In [1]:
import deepchem as dc
from openpom.feat.graph_featurizer import GraphFeaturizer, GraphConvConstants
from openpom.utils.data_utils import get_class_imbalance_ratio, IterativeStratifiedSplitter
from openpom.models.mpnn_pom import MPNNPOMModel
from datetime import datetime
from tqdm import tqdm
import torch
import numpy as np
from sklearn.metrics import roc_auc_score

Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (/home/ary2260/miniconda3/envs/testenv/lib/python3.9/site-packages/deepchem/models/torch_models/__init__.py)
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'pytorch_lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'


In [3]:
TASKS = [
'alcoholic', 'aldehydic', 'alliaceous', 'almond', 'amber', 'animal',
'anisic', 'apple', 'apricot', 'aromatic', 'balsamic', 'banana', 'beefy',
'bergamot', 'berry', 'bitter', 'black currant', 'brandy', 'burnt',
'buttery', 'cabbage', 'camphoreous', 'caramellic', 'cedar', 'celery',
'chamomile', 'cheesy', 'cherry', 'chocolate', 'cinnamon', 'citrus', 'clean',
'clove', 'cocoa', 'coconut', 'coffee', 'cognac', 'cooked', 'cooling',
'cortex', 'coumarinic', 'creamy', 'cucumber', 'dairy', 'dry', 'earthy',
'ethereal', 'fatty', 'fermented', 'fishy', 'floral', 'fresh', 'fruit skin',
'fruity', 'garlic', 'gassy', 'geranium', 'grape', 'grapefruit', 'grassy',
'green', 'hawthorn', 'hay', 'hazelnut', 'herbal', 'honey', 'hyacinth',
'jasmin', 'juicy', 'ketonic', 'lactonic', 'lavender', 'leafy', 'leathery',
'lemon', 'lily', 'malty', 'meaty', 'medicinal', 'melon', 'metallic',
'milky', 'mint', 'muguet', 'mushroom', 'musk', 'musty', 'natural', 'nutty',
'odorless', 'oily', 'onion', 'orange', 'orangeflower', 'orris', 'ozone',
'peach', 'pear', 'phenolic', 'pine', 'pineapple', 'plum', 'popcorn',
'potato', 'powdery', 'pungent', 'radish', 'raspberry', 'ripe', 'roasted',
'rose', 'rummy', 'sandalwood', 'savory', 'sharp', 'smoky', 'soapy',
'solvent', 'sour', 'spicy', 'strawberry', 'sulfurous', 'sweaty', 'sweet',
'tea', 'terpenic', 'tobacco', 'tomato', 'tropical', 'vanilla', 'vegetable',
'vetiver', 'violet', 'warm', 'waxy', 'weedy', 'winey', 'woody'
]

print("No of tasks: ", len(TASKS))
n_tasks = len(TASKS)

No of tasks:  138


save train and test splits

In [4]:
# uncomment and run if no splits saved yet

# download curated dataset
# !wget https://raw.githubusercontent.com/ARY2260/openpom/main/openpom/data/curated_datasets/curated_GS_LF_merged_4983.csv

# The curated dataset can also found at `openpom/data/curated_datasets/curated_GS_LF_merged_4983.csv` in the repo.

input_file = 'curated_GS_LF_merged_4983.csv' # or new downloaded file path

# get dataset

featurizer = GraphFeaturizer()
smiles_field = 'nonStereoSMILES'
loader = dc.data.CSVLoader(tasks=TASKS,
                   feature_field=smiles_field,
                   featurizer=featurizer)
dataset = loader.create_dataset(inputs=[input_file])
n_tasks = len(dataset.tasks)

# get k folds list
k = 5
splitter = IterativeStratifiedSplitter(order=2)
directories = ['']*2*k
for fold in range(k):
    directories[2 * fold] = f'./ensemble_cv_exp/fold_{fold+1}/train_data'
    directories[2 * fold + 1] = f'./ensemble_cv_exp/fold_{fold+1}/cv_data'
folds_list = splitter.k_fold_split(dataset=dataset, k=k, directories=directories)

ensemble function

In [6]:
def benchmark_ensemble(fold, train_dataset, test_dataset, n_models, nb_epoch):
    train_ratios = get_class_imbalance_ratio(train_dataset)
    assert len(train_ratios) == n_tasks

    # learning_rate = 0.001
    learning_rate = dc.models.optimizers.ExponentialDecay(initial_rate=0.001, decay_rate=0.5, decay_steps=32*20, staircase=True)
    metric = dc.metrics.Metric(dc.metrics.roc_auc_score)

    # fit models
    for i in tqdm(range(n_models)):
        model = MPNNPOMModel(n_tasks = n_tasks,
                                batch_size=128,
                                learning_rate=learning_rate,
                                class_imbalance_ratio = train_ratios,
                                loss_aggr_type = 'sum',
                                node_out_feats = 100,
                                edge_hidden_feats = 75,
                                edge_out_feats = 100,
                                num_step_message_passing = 5,
                                mpnn_residual = True,
                                message_aggregator_type = 'sum',
                                mode = 'classification',
                                number_atom_features = GraphConvConstants.ATOM_FDIM,
                                number_bond_features = GraphConvConstants.BOND_FDIM,
                                n_classes = 1,
                                readout_type = 'set2set',
                                num_step_set2set = 3,
                                num_layer_set2set = 2,
                                ffn_hidden_list= [392, 392],
                                ffn_embeddings = 256,
                                ffn_activation = 'relu',
                                ffn_dropout_p = 0.12,
                                ffn_dropout_at_input_no_act = False,
                                weight_decay = 1e-5,
                                self_loop = False,
                                optimizer_name = 'adam',
                                log_frequency = 32,
                                model_dir = f'./ensemble_cv_exp/ensemble_fold_{fold+1}/experiments_{i+1}',
                                device_name='cuda')

        start_time = datetime.now()
        
        # fit model
        loss = model.fit(
            train_dataset,
            nb_epoch=nb_epoch,
            max_checkpoints_to_keep=1,
            deterministic=False,
            restore=False)
        end_time = datetime.now()
        
        train_scores = model.evaluate(train_dataset, [metric])['roc_auc_score']
        test_scores = model.evaluate(test_dataset, [metric])['roc_auc_score']
        print(f"loss = {loss}; train_scores = {train_scores}; test_scores = {test_scores}; time_taken = {str(end_time-start_time)}")
        model.save_checkpoint() # saves final checkpoint => `checkpoint2.pt`
        del model
        torch.cuda.empty_cache()    

    # Get test score from the ensemble
    list_preds = []
    for i in range(n_models):
        model = MPNNPOMModel(n_tasks = n_tasks,
                                batch_size=128,
                                learning_rate=learning_rate,
                                class_imbalance_ratio = train_ratios,
                                loss_aggr_type = 'sum',
                                node_out_feats = 100,
                                edge_hidden_feats = 75,
                                edge_out_feats = 100,
                                num_step_message_passing = 5,
                                mpnn_residual = True,
                                message_aggregator_type = 'sum',
                                mode = 'classification',
                                number_atom_features = GraphConvConstants.ATOM_FDIM,
                                number_bond_features = GraphConvConstants.BOND_FDIM,
                                n_classes = 1,
                                readout_type = 'set2set',
                                num_step_set2set = 3,
                                num_layer_set2set = 2,
                                ffn_hidden_list= [392, 392],
                                ffn_embeddings = 256,
                                ffn_activation = 'relu',
                                ffn_dropout_p = 0.12,
                                ffn_dropout_at_input_no_act = False,
                                weight_decay = 1e-5,
                                self_loop = False,
                                optimizer_name = 'adam',
                                log_frequency = 32,
                                model_dir = f'./ensemble_cv_exp/ensemble_fold_{fold+1}/experiments_{i+1}',
                                device_name='cuda')
        model.restore(f"./ensemble_cv_exp/ensemble_fold_{fold+1}/experiments_{i+1}/checkpoint2.pt")
        # test_scores = model.evaluate(test_dataset, [metric])['roc_auc_score']
        # print("test_score: ", test_scores)
        preds = model.predict(test_dataset)
        list_preds.append(preds)

    preds_arr = np.asarray(list_preds)
    ensemble_preds = np.mean(preds_arr, axis=0)
    return roc_auc_score(test_dataset.y, ensemble_preds, average="macro")

k-fold ensemble cv

In [7]:
n_models = 10
nb_epoch = 62
folds_results = []
for fold in tqdm(range(k)):
    print(f"Fold {fold+1} ensemble starting now.")
    train_dataset = dc.data.DiskDataset(directories[2 * fold])
    test_dataset = dc.data.DiskDataset(directories[2 * fold + 1])
    print("train_dataset: ", len(train_dataset))
    print("test_dataset: ", len(test_dataset))
    fold_result = benchmark_ensemble(fold=fold,
                                     train_dataset=train_dataset,
                                     test_dataset=test_dataset,
                                     n_models=n_models,
                                     nb_epoch=nb_epoch)
    print(f"Fold {fold+1} ensemble score: ", fold_result)
    folds_results.append(fold_result)

  0%|          | 0/5 [00:00<?, ?it/s]

Fold 1 ensemble starting now.
train_dataset:  3994
test_dataset:  989


loss = 1.602946400642395; train_scores = 0.9573782936182699; test_scores = 0.8619264719732461; time_taken = 0:03:22.696741


loss = 1.7123421430587769; train_scores = 0.9516135507806346; test_scores = 0.8679550912444497; time_taken = 0:03:28.300721


loss = 1.7177332639694214; train_scores = 0.9527891970355443; test_scores = 0.8698497855865827; time_taken = 0:03:25.572594


...

loss = 1.5471675395965576; train_scores = 0.958369011186212; test_scores = 0.8716890642086778; time_taken = 0:03:21.230315


100%|██████████| 10/10 [33:51<00:00, 203.16s/it]

loss = 1.6139239072799683; train_scores = 0.9571089217390171; test_scores = 0.8729007628769709; time_taken = 0:03:21.375321



100%|██████████| 5/5 [2:51:40<00:00, 2060.10s/it]

Fold 5 ensemble score:  0.8897389680117992


In [10]:
cv_mean_result = np.mean(folds_results)

with open("./ensemble_cv_exp/final_score.txt", 'w+') as f:
    f.write(f"folds_results = {folds_results}\n")
    f.write(f"cv_mean_result = {cv_mean_result}\n")

In [13]:
round(cv_mean_result, 4)

0.8872